# Run model

This runs the stun model as defined in a Stan file provided, for the 5 environmental variables used as predictors, and fraction of the cultivated area used as a response variable. It assumes that the model is located in the 'stan_models' directory.

## Variables to be defined:

*@mname - name of a model  
@warmup - number of warmup iterations  
@iter - overall number of iterations  
@chains - number of chains  
@corese - number of cores  
@thin - thin*  

In [1]:
### Set working directory:
setwd("/export/home/mk843/mnt/")
#setwd("G:\\My Drive\\SDM_China\\")

### Define the model to run
mname<-'parabolic_interactions_iCAR'
stan_model<-paste('stan_models/',mname,'.stan',sep="")

### Define model parameters:

warmup=2000
iter=10000
chains=5
cores=5
thin=1

### Define the path in which to save models:
path2models<-"outputs/models/"

## Load libraries and data

In [2]:
### Load libraries
library(rstan)
library(rgdal)

### Load data
china<-readOGR(dsn = "data/china_data", layer = "china_data")
### Get the data on counties adjacency
adjacency<-read.csv("data/china_data/adjacency.csv")

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.19.2, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

For improved execution time, we recommend calling
Sys.setenv(LOCAL_CPPFLAGS = '-march=native')
although this causes Stan to throw an error on a few processors.

Loading required package: sp

rgdal: version: 1.4-4, (SVN revision 833)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.2.3, released 2017/11/20
 Path to GDAL shared files: C:/Users/ycrnm/Documents/R/win-library/3.5/rgdal/gdal
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: C:/Users/ycrnm/Documents/R/win-library/3.5/rgdal/proj
 Linking to sp versi

OGR data source with driver: ESRI Shapefile 
Source: "G:\My Drive\SDM_China\data\china_data", layer: "china_data"
with 2409 features
It has 36 fields


## Format data

In [5]:
### Get the adjacent nodes from the matrix:
node1<-adjacency$node1
node2<-adjacency$node2
N_edges<-length(node1)
N<-nrow(china@data)

# Defince stan data:
y=china@data$logArea
x1=china@data$BIO10_sd
x2=china@data$BIO17_sd
x3=china@data$BIO4_sd
x4=china@data$BIO9_sd
x5=china@data$npp_sd

#Format the data
stan_data <- list(N = N, x1 = x1, y = y,x2=x2,x3=x3,x4=x4,x5=x5,N_edges=N_edges,node1=node1,node2=node2)

## Run and save model

In [ ]:
print("Fitting the model")

fit <- stan(file = stan_model, data = stan_data, warmup = warmup, iter = iter, chains = chains, cores = cores, thin = thin,control = list(max_treedepth = 15))

print("Saving the model")
saveRDS(fit, paste(path2models, mname,".rds",sep="")) 